Lab3 is based on highway-env https://github.com/eleurent/highway-env. In
envs/highway_env.py, the vehicle is rewarded for reaching a high speed, staying on the rightmost lanes and avoiding collisions for in highway driving (highway-env). Your task is to add an additional term to penalize changing lanes. (Ideally we should penalize changing lanes to the left more than changing to the right, but this is a simplified exercise.) Please refer to envs/roundabout_env for reference.

Steps:

1) Fork the highway-env repository to your own GitHub account, and modify highway_env.py. (if you are not familiar with GitHub, I recommend installing GitHub Desktop https://desktop.github.com.)

2) Replace
!pip install highway-env
with path to your own repository:
!pip install git+https://github.com/USERNAME/highway-env

3) Train your agent for different values of lane_change_reward, and different environments with varying complexity, starting from the simplest:

env.config["lane_change_reward"] = 0

env.config["vehicles_count"] = 0

env.config["lanes_count"] = 2

env.config["initial_lane_id"] = 0

In this simple environment, during testing the vehicle should switch to the right lane quickly and stay in the right lane. (It is possible for it to stay in the left lane during testing if your training num_episodes is too small, since the agent may not have had a chance to try the lane change action during training.)
Try different values of lane_change_reward (e.g., -0.1, -1, -10, etc. note that it must be negative), and different numbers of vehicles/lanes. Make lane_change_reward very negative until you observe that the vehicle prefers *collision* to *lane change to avoid collision*. Submit your IPython Notebook, and a short report (or text blocks within the Notebook) describing the different configurations you have tried, and your observations for each config, including two configs that differ only in the lane_change_reward, one with no or rare collisions, one with frequent collisions.


You may use either Google CoLab or your local machine. If you use CoLab, please send your CoLab web link. Otherwise, please submit your IPython Notebook to Canvas. Please include the recorded videos during the testing in IPython Notebook. Since it is not possible to save multiple trained models without modifying the source code of rl-agents, we can only see one trained model in Colab that you trained last. It is not realistic for us to retrain all the models, so please provide screenshots and descriptions in the report for your observations of each config, esp. "the two configs that differ only in the lane_change_reward, one with no or rare collisions, one with frequent collisions".

Notes:

1) More complex environments demand more training time (more num_episodes), so do not make your environment more complex than necessary. You may try increasing the  vehicles_density and reward_speed_range to increase the chances of collision. You may also find "Script to Stop Google Colab From Disconnecting" to be helpful.

2) You may want to tune DQN hyperparams in configs/HighwayEnv/agents/DQNAgent/dqn.json, e.g., discount factor gamma, and epsilon greedy exploration params tau, temperature, final_temperature for different num_episodes.

3) The error message "fatal: destination path 'highway-env' already exists and is not an empty directory" in the first code cell can be safely ignored.

Tips (2024/01):
1. To open a Python file from GitHub in Colab, simply change the domain from 'github.com' to 'githubtocolab.com’, e.g.,
https://githubtocolab.com/guzonghua/saavlabs/blob/main/Lab3/Lab3_Highway_DQN_rlagents.ipynb
Then you can click on “Copy to Drive” to copy it to your Google Drive.

2. If you encounter this error “module 'collections' has no attribute 'Mapping’”
https://www.perplexity.ai/search/module-collections-has-QYCyz9XNQBaNwVo5QhA3jA?s=u

Please go to the rl_agents library  (where it is installed) and editing the file  configuration.py:

Change line 1:  import collections -> import collections.abc

Change line 39:  if isinstance(v, collections.Mapping): -> if isinstance(v, collections.abc.Mapping):

# Training a DQN on `highway-v0` with rlagents
## Import requirements

In [2]:
# Environment

## KP-test version 2022-12-09 with working video

!pip install highway-env ##KP
!pip install --user git+https://github.com/eleurent/highway-env
##KP - install newer version than published - clone from that
!pip install git+https://github.com/bogrundtman/highway-env
import gym
import highway_env

# Agent
##KP !pip install git+https://github.com/eleurent/rl-agents
!pip install --user git+https://github.com/eleurent/rl-agents
!pip install moviepy ##KP - used for playing videos
#from moviepy.editor import *  #KP - gives error

# Visualisation utils
import sys
%load_ext tensorboard
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!git clone https://github.com/eleurent/highway-env.git
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import record_videos,show_videos
!pip install imageio-ffmpeg #KP - mpeg4 encoder needed for play
import imageio_ffmpeg
#from moviepy.video.io.ImageSequenceClip import ImageSequenceClip ##KP

  Cloning https://github.com/eleurent/highway-env to /tmp/pip-req-build-ycanba_p
  Running command git clone --filter=blob:none --quiet https://github.com/eleurent/highway-env /tmp/pip-req-build-ycanba_p
  Resolved https://github.com/eleurent/highway-env to commit 4cae2587999ac342a7b5c34507af8d20397a163f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/bogrundtman/highway-env to /tmp/pip-req-build-i559bv4_
  Running command git clone --filter=blob:none --quiet https://github.com/bogrundtman/highway-env /tmp/pip-req-build-i559bv4_
  Resolved https://github.com/bogrundtman/highway-env to commit 4cae2587999ac342a7b5c34507af8d20397a163f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


ModuleNotFoundError: No module named 'highway_env'

## Training

Prepare environment, agent, and evaluation process.


In [ ]:
from rl_agents.trainer.evaluation import Evaluation
from rl_agents.agents.common.factory import load_agent, load_environment

# Get the environment and agent configurations from the rl-agents repository
!git clone https://github.com/eleurent/rl-agents.git
%cd /content/rl-agents/scripts/
env_config = 'configs/HighwayEnv/env.json'
agent_config = 'configs/HighwayEnv/agents/DQNAgent/dqn.json'

env = load_environment(env_config)
import pprint
from matplotlib import pyplot as plt
env.config["lane_change_reward"] = 0
env.config["vehicles_count"] = 0
#env.config['reward_speed_range'] = [0, 100]
env.config["lanes_count"] = 2
env.config["initial_lane_id"] = 0
pprint.pprint(env.config)
pprint.pprint(agent_config)

env.reset()
plt.imshow(env.render(mode="rgb_array"))
plt.show()

agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=10, display_env=False)
print(f"Ready to train {agent} on {env}")

Start training. Run tensorboard locally to visualize training.

In [ ]:
%tensorboard --logdir "{evaluation.directory}"
evaluation.train()

Progress can be visualised in the tensorboard cell above, which should update every 30s (or manually). You may need to click the *Fit domain to data* buttons below each graph.

## Testing

Run the learned policy for a few episodes.

In [ ]:
#env = load_environment(env_config)
env.configure({"offscreen_rendering": True})
env.render(mode="rgb_array")
##env = record_videos(env)
agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=3, recover=True)
evaluation.test()
show_videos(evaluation.run_directory)